Data Prep NLP

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.corpus import stopwords
from langdetect import detect



In [7]:
# import nltk
# nltk.download('stopwords')
# help(pd)


In [526]:
%pwd

'C:\\Users\\geek\\git\\OCT25_BMLE_RAKUTEN\\notebooks'

In [527]:
from print_versions import print_versions
print_versions(globals())

pandas==2.3.3
numpy==2.3.5
nltk==3.9.2
langdetect==1.0.9
pycountry==24.6.1
translate==3.8.0
libretranslatepy==2.1.1


In [597]:
##import original data
##data is already split into Training/Testing, no need to re-split
Rak_train_raw = pd.read_csv('../data/raw/X_train_update.csv', index_col=0)
RakY_train_raw = pd.read_csv('../data/raw/Y_train_CVw08PX.csv', index_col=0)
Rak_test_raw = pd.read_csv('../data/raw/X_test_update.csv', index_col=0)


In [511]:
print(pd.get_option('max_colwidth'))  ##50
pd.set_option('max_colwidth', 0)

display(Rak_train_raw.head()) 


print(Rak_train_raw.info())
print(Rak_train_raw.describe())
print(Rak_train_raw.isna().sum())

print("Train/Test split: ", len(Rak_train_raw) / (len(Rak_train_raw + RakX_test_raw)))

##FIXME problematic strings for clean-up
## &#39;
## àªtre
## &#34;
## </li><li>
## &#34
## &agrave
## &eacute
## L', d'
##FIXME http://www (any non ' ' string between htttp and .com or http and .html)
##FIXME identify german strings / charsg
##FIXME more advanced approach: could use LLM to identify text language, and apply keywords selectively

##FIXME do replace problematic strings with dict
# Rak_train_prep = Rak_train_raw.replace()



0


,designation,description,productid,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seiten / Punktraster / Ca Din A5 / Rosen-Design,NaN,3804725264,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L'art Et Son Marche Salon D'art Asiatique A Paris - Jacques Barrere - Francois Perrier - La Reforme Des Ventes Aux Encheres Publiques - Le Sna Fete Ses Cent Ans.,NaN,436067568,1008141237
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo Wii U - Speedlink Pilot Style,"PILOT STYLE Touch Pen de marque Speedlink est 1 stylet ergonomique pour GamePad Nintendo Wii U.<br> Pour un confort optimal et une précision maximale sur le GamePad de la Wii U: ce grand stylet hautement ergonomique est non seulement parfaitement adapté à votre main mais aussi très élégant.<br> Il est livré avec un support qui se fixe sans adhésif à l'arrière du GamePad<br> <br> Caractéristiques:<br> Modèle: Speedlink PILOT STYLE Touch Pen<br> Couleur: Bleu<br> Ref. Fabricant: SL-3468-BE<br> Compatibilité: GamePad Nintendo Wii U<br> Forme particulièrement ergonomique excellente tenue en main<br> Pointe à revêtement longue durée conçue pour ne pas abîmer l'écran tactile<br> En bonus : Support inclu pour GamePad<br> <span class=""vga_style2""><b></b><br>",201115110,938777978
3,Peluche Donald - Europe - Disneyland 2000 (Marionnette À Doigt),NaN,50418756,457047496
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut organiser un jeu de guerre de boules de neige et s'arranger pour en &ecirc;tre le vainqueur incontest&eacute;. Mais Sophie s'en m&ecirc;le et chambarde tous ses plans...,278535884,1077757786


<class 'pandas.core.frame.DataFrame'>
Index: 84916 entries, 0 to 84915
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   designation  84916 non-null  object
 1   description  55116 non-null  object
 2   productid    84916 non-null  int64 
 3   imageid      84916 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 5.3+ MB
None
          productid       imageid
count  8.491600e+04  8.491600e+04
mean   2.555468e+09  1.152691e+09
std    1.588656e+09  1.751427e+08
min    1.839120e+05  6.728400e+04
25%    6.760519e+08  1.056269e+09
50%    3.190506e+09  1.213354e+09
75%    3.995599e+09  1.275646e+09
max    4.252012e+09  1.328824e+09
designation    0    
description    29800
productid      0    
imageid        0    
dtype: int64
Train/Test split:  0.8601004780811927


In [310]:
##Display RAW data (random rows)
Xa_raw = Rak_train_raw.sample(10)
display(Xa_raw) 
# print(Xa_raw.index)

designation  \
64317  Réaliste Reborn 55cm Reborn Lifelike En Vinyle Souple En Silicone Noël Présent                                                 
17312  Carnet De Notes Bloc-Notes Cahier L'usine De Peinture Orange Fleur Verte Carnet De Journal Journal A5 De La Chimie Classique   
4659   Carnet De Notes Bloc-Notes Cahier Muet En Nou Journal A5                                                                       
46812  Staedtler Lot De 12 250 05-3h Recharges Pour Porte-Mine Noir                                                                   
82728  Foil Premium Shinobi Du Gang Okiba  - Magic Mtg - Traitres De Kamigawa 76 - C                                                  
11654  Bureau Halloween Décoration D'intérieur Canapé Design Géométrique Coussin Carré Coussin Co_Hoeroie6865                         
6099   Monsieur Bébé ® Tour Et Fond De Parc Molletonné Pour Parc Évolutif Monsieur Bébé 70 X 95 Cm - Fond Marron - Étoiles Blanches   
10173  Poignées De Traction Bandes De Résistance Remplacement De Mousse De Remise En Forme Noir Pour L'exercice Yoga Stream           
54864  7.4v 1500mah Battery Spare Part For Wltoys V913/L959/Wl912 Rc Helicopterzen5752-Générique                                      
2359   Rexel Auto+ 90x - Destructeur De Documents - Coupe Croisée - 4 X 45 Mm - P-3                                                   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [299]:
##FIXME hard to fix edge cases
# display(Rak_train_raw.iloc[[56642]])   ##FIXME corrupted fr accents
# display(Rak_train_raw.iloc[[63384]])   ##FIXME mangled words
# display(Rak_train_raw.iloc[[3935, 26649, 18102, 42210]])   ##FIXME edge cases


**Clean-up strings**

In [302]:
##test unescape approaches
from bs4 import BeautifulSoup

# Sample string with HTML tags
s1 = "<h1>Welcome to <b>Python Programming</b></h1>"

# Removing HTML tags using Beautiful Soup
soup = BeautifulSoup(s1, "html.parser")
s2 = soup.get_text()
# print(s2)

# help(BeautifulSoup)

import html
# print(html.unescape('asdf &pound;682m asdf &agrave asdf &#39; &amp; &#34; &#43;'))

x = pd.DataFrame({'a': ['asdf &pound;682m asdf &agrave asdf &#39; &amp; &#34; &#43;', 'asdf &pound;682m asdf &agrave asdf &#39; &amp; &#34; &#43;']})
# display(x)

    # print(html.unescape(x['a']))
# print(help(pd.DataFrame.apply))

# display(x.apply(lambda row: html.unescape(row['a']), axis = 1))


In [145]:
##Fix problematic strings for text clean-up

Rak_train = Rak_train_raw.copy(deep = True)

## Replace NaN with ''
## (for some reason strings can include numeric NaN values)
Rak_train['description'] = Rak_train['description'].fillna('')

## lower case
Rak_train['designation'] = Rak_train['designation'].str.lower()
Rak_train['description'] = Rak_train['description'].str.lower()

##FIXME simplify syntax
##Unescape HTML
Rak_train['designation'] = Rak_train.apply(lambda row: html.unescape(row['designation']), axis = 1)
Rak_train['description'] = Rak_train.apply(lambda row: html.unescape(row['description']), axis = 1)

## Remove HTML tags
Rak_train['designation'] = Rak_train.apply(lambda row: BeautifulSoup(row['designation'], "html.parser").get_text(separator=" "), axis = 1)
Rak_train['description'] = Rak_train.apply(lambda row: BeautifulSoup(row['description'], "html.parser").get_text(separator=" "), axis = 1)

##FIXME drop empty designation & description rows
##FIXME make fun to clean-up data (to apply to test as well)


C:\Users\geek\AppData\Local\Temp\ipykernel_32460\589964789.py:21: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  Rak_train['description'] = Rak_train.apply(lambda row: BeautifulSoup(row['description'], "html.parser").ge

In [146]:

##FIXME list of problematic strings to fix
## àªtre

##Regex replacements
repl_dict = {
             ##FIXME nltk.classify.textcat.TextCat().remove_punctuation() 
             r'n°': r' numéro ', 

             ##FIXME not sure how to handle '¿' or '?'
             ##insert space around any non-digit, non-word and non-whitespace with (e.g. '\?' -> ' \? ', 'n°' -> 'n ° ')
             ##except ¿'
             r"[^\d\w\s¿\?'\-]": r' \g<0> ',  

             ##FIXME possibly remove digits after translation
             r"\b\S*[0-9]+\S*\b": '' ##drop all words that contain digits (so drop all digits as well)
             # '&#39;': ' ', 
             # '&#34;': ' ',
             # '&#43;': ' ',
             # '&amp;': ' ',
             # '&.{3};': ' ', 
             # '<li>': ' ',
             # '</li>': ' ',
             # '<p>': ' ',
             # '</p>': ' ',
             # '<br>': ' ',
             # '</br>': ' ',
             # '<br />': ' ',
             # '<b>': ' ',
             # '</b>': ' ',
             # '<ul>': ' ',
             # '</ul>': ' ',
             # '<strong>': ' ',
             # '</strong>': ' ',
             # '<span.*>': ' ',
             # '&agrave': ' ',
             # r'\?': ' ',  ##replace '?' with ' ' 
             # ':': ' ',
             # '®': ' ', 
             # r'n°': r'n° ', 
             # r'\)': r'\) '
            }
# print({'var': repl_dict})

Rak_train = Rak_train.replace(to_replace = {'designation': repl_dict,
                                               'description': repl_dict},
                                 regex = True)


In [591]:
##Clean-up strings
def rak_data_cleanup(rak_data_raw):
    ##Fix problematic strings for text clean-up
    ##FIXME not sure if deep copy inside function is needed
    rak_data = rak_data_raw.copy(deep = True)

    ## Replace NaN with ''
    ## (for some reason strings can include numeric NaN values)
    rak_data['description'] = rak_data['description'].fillna('')

    ## lower case
    rak_data['designation'] = rak_data['designation'].str.lower()
    rak_data['description'] = rak_data['description'].str.lower()

    ##FIXME simplify syntax
    ##Unescape HTML
    rak_data['designation'] = rak_data.apply(lambda row: html.unescape(row['designation']), axis = 1)
    rak_data['description'] = rak_data.apply(lambda row: html.unescape(row['description']), axis = 1)

    ## Remove HTML tags
    rak_data['designation'] = rak_data.apply(lambda row: BeautifulSoup(row['designation'], "html.parser").get_text(separator=" "), axis = 1)
    rak_data['description'] = rak_data.apply(lambda row: BeautifulSoup(row['description'], "html.parser").get_text(separator=" "), axis = 1)

    ##FIXME drop empty designation & description rows
    ##FIXME make fun to clean-up data (to apply to test as well)


    ##FIXME list of problematic strings to fix
    ## àªtre

    ##Regex replacements
    repl_dict = {
                ##FIXME nltk.classify.textcat.TextCat().remove_punctuation() 
                r'n°': r' numéro ', 

                ##FIXME not sure how to handle '¿' or '?'
                ##insert space around any non-digit, non-word and non-whitespace with (e.g. '\?' -> ' \? ', 'n°' -> 'n ° ')
                ##except ¿'
                r"[^\d\w\s¿\?'\-]": r' \g<0> ',  

                ##FIXME possibly remove digits after translation
                r"\b\S*[0-9]+\S*\b": '' ##drop all words that contain digits (so drop all digits as well)
                }

    rak_data = rak_data.replace(to_replace = {'designation': repl_dict,
                                                'description': repl_dict},
                                    regex = True)



    ##Concat strings
    rak_data['product_txt'] = rak_data['designation'] + ' . -//- ' + rak_data['description']

    rak_data['product_txt_len'] = rak_data['product_txt'].apply(len)

    return rak_data



In [613]:
##clean-up X train data
Rak_train = rak_data_cleanup(Rak_train_raw)


C:\Users\geek\AppData\Local\Temp\ipykernel_32460\527297690.py:22: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  rak_data['description'] = rak_data.apply(lambda row: BeautifulSoup(row['description'], "html.parser").get_

In [624]:
display(Rak_train.head())

print(Rak_train.shape)

,Unnamed: 0,designation,description,productid,imageid,lang,product_txt,product_tokens,product_txt_len,product_txt_transl
0,0,olivia : personalisiertes notizbuch / seiten / punktraster / ca din / rosen-design,NaN,3804725264,1263597046,de,olivia : personalisiertes notizbuch / seiten / punktraster / ca din / rosen-design . -//-,"['olivia', 'personalisiertes', 'notizbuch', 'seiten', 'punktraster', 'din', 'rosen', 'design']",101,olivia : notizbuch / page / punktraster / ca din / rosen-design
1,1,journal des arts ( le ) n ° du / / - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans .,NaN,436067568,1008141237,fr,journal des arts ( le ) n ° du / / - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans . . -//-,"['journal', 'des', 'arts', 'art', 'son', 'marche', 'salon', 'art', 'asiatique', 'paris', 'jacques', 'barrere', 'francois', 'perrier', 'reforme', 'des', 'ventes', 'aux', 'encheres', 'publiques', 'sna', 'fete', 'ses', 'cent', 'ans']",215,journal des arts ( le ) n ° du / / - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans . . -//-
2,2,grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style,pilot style touch pen de marque speedlink est stylet ergonomique pour gamepad nintendo wii u . pour un confort optimal et une précision maximale sur le gamepad de la wii u : ce grand stylet hautement ergonomique est non seulement parfaitement adapté à votre main mais aussi très élégant . il est livré avec un support qui se fixe sans adhésif à l'arrière du gamepad caractéristiques : modèle : speedlink pilot style touch pen couleur : bleu ref . fabricant : compatibilité : gamepad nintendo wii u forme particulièrement ergonomique excellente tenue en main pointe à revêtement longue durée conçue pour ne pas abîmer l'écran tactile en bonus : support inclu pour gamepad,201115110,938777978,fr,grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style . -//- pilot style touch pen de marque speedlink est stylet ergonomique pour gamepad nintendo wii u . pour un confort optimal et une précision maximale sur le gamepad de la wii u : ce grand stylet hautement ergonomique est non seulement parfaitement adapté à votre main mais aussi très élégant . il est livré avec un support qui se fixe sans adhésif à l'arrière du gamepad caractéristiques : modèle : speedlink pilot style touch pen couleur : bleu ref . fabricant : compatibilité : gamepad nintendo wii u forme particulièrement ergonomique excellente tenue en main pointe à revêtement longue durée conçue pour ne pas abîmer l'écran tactile en bonus : support inclu pour gamepad,"['grand', 'stylet', 'ergonomique', 'bleu', 'gamepad', 'nintendo', 'wii', 'speedlink', 'pilot', 'style', 'pilot', 'style', 'touch', 'pen', 'marque', 'speedlink', 'est', 'stylet', 'ergonomique', 'pour', 'gamepad', 'nintendo', 'wii', 'pour', 'confort', 'optimal', 'une', 'précision', 'maximale', 'sur', 'gamepad', 'wii', 'grand', 'stylet', 'hautement', 'ergonomique', 'est', 'non', 'seulement', 'parfaitement', 'adapté', 'votre', 'main', 'mais', 'aussi', 'très', 'élégant', 'est', 'livré', 'avec', 'support', 'qui', 'fixe', 'sans', 'adhésif', 'arrière', 'gamepad', 'caractéristiques', 'modèle', 'speedlink', 'pilot', 'style', 'touch', 'pen', 'couleur', 'bleu', 'ref', 'fabricant', 'compatibilité', 'gamepad', 'nintendo', 'wii', 'forme', 'particulièrement', 'ergonomique', 'excellente', 'tenue', 'main', 'pointe', 'revêtement', 'longue', 'durée', 'conçue', 'pour', 'pas', 'abîmer', 'écran', 'tactile', 'bonus', 'support', 'inclu', 'pour', 'gamepad']",781,grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style . -//- pilot style touch pen de marque speedlink est stylet ergono

(84916, 10)


In [615]:
##clean-up X test data
Rak_test = rak_data_cleanup(Rak_test_raw)


C:\Users\geek\AppData\Local\Temp\ipykernel_32460\527297690.py:22: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  rak_data['description'] = rak_data.apply(lambda row: BeautifulSoup(row['description'], "html.parser").get_

In [619]:
display(Rak_test.head())

print(Rak_test.shape)

,designation,description,productid,imageid,product_txt,product_txt_len,lang
84916,folkmanis puppets - - marionnette et théâtre - mini turtle,,516376098,1019294171,folkmanis puppets - - marionnette et théâtre - mini turtle . -//-,67,fr
84917,porte flamme gaxix - flamebringer gaxix - / - u - twilight of the dragons,,133389013,1274228667,porte flamme gaxix - flamebringer gaxix - / - u - twilight of the dragons . -//-,83,en
84918,pompe de filtration speck badu,,4128438366,1295960357,pompe de filtration speck badu . -//-,39,fr
84919,robot de piscine électrique,ce robot de piscine d'un design innovant et élégant apportera un nettoyage efficace et rapide . il est conçu pour tous types de revêtements tels que le vinyle le béton la fibre de verre et la céramique . avec un dispositif de commande il est facile à régler le cycle de nettoyage et le temps . il suffit de brancher ce robot et le mettre dans la piscine . ses deux filtres à paniers intérieurs sont mobiles pour retenir les moindres débits et le nettoyage . ce robot de piscine robuste d'une vitesse de gal / h est parfait pour nettoyer la totalité de votre piscine . avec son boîtier en thermoplaste ce robot de piscine vous donne une belle longévité . robot de piscine convient aux piscines de tous types contrôlé par l'ordinateur programme de nettoyage et de filtration mur d'escalade longueur du câble : m vitesse : gal / h protection : puissance : w cycle de nettoyage : / / / s durée du cycle : / / h opération silencieuse facile à utiliser,3929899732,1265224052,robot de piscine électrique . -//- ce robot de piscine d'un design innovant et élégant apportera un nettoyage efficace et rapide . il est conçu pour tous types de revêtements tels que le vinyle le béton la fibre de verre et la céramique . avec un dispositif de commande il est facile à régler le cycle de nettoyage et le temps . il suffit de brancher ce robot et le mettre dans la piscine . ses deux filtres à paniers intérieurs sont mobiles pour retenir les moindres débits et le nettoyage . ce robot de piscine robuste d'une vitesse de gal / h est parfait pour nettoyer la totalité de votre piscine . avec son boîtier en thermoplaste ce robot de piscine vous donne une belle longévité . robot de piscine convient aux piscines de tous types contrôlé par l'ordinateur programme de nettoyage et de filtration mur d'escalade longueur du câble : m vitesse : gal / h protection : puissance : w cycle de nettoyage : / / / s durée du cycle : / / h opération silencieuse facile à utiliser,1016,fr
84920,hsm destructeur securio coupe crois¿e : x mm,,152993898,940543690,hsm destructeur securio coupe crois¿e : x mm . -//-,56,fr


(13812, 7)


In [616]:
##detect phrase language using 'langid' on product_txt
def lang_detect(data):
    # langid.set_languages(langs=None)
    langid.set_languages(langs=['fr', 'en', 'de', 'it', 'es', 'pt'])
    data['lang'] = data['product_txt'].apply(lambda x: langid.classify(x)[0])

    return data


In [617]:
##detect languages on Rakuten processed data
Rak_train = lang_detect(data = Rak_train)
Rak_test = lang_detect(data = Rak_test)


In [622]:
##translate using libretranslate (self-hosted process)
##func to obtain translations 
## if rerun == True then run translation code, if False (default) then just load csv from supplied path
def translate_txt(data = None, rerun = False, csv_file = '../data/processed/Rak_train_translations.csv'):
    ##FIXME improve func not to require data if just loading csv
    if rerun == True:
        ##NOTE: need to start external process first (and this code is time-consuming)
        # libretranslate --update-models --load-only fr,en,es,de,it,pt
        # libretranslate --load-only fr,en,es,de,it,pt
        lt = LibreTranslateAPI("http://localhost:5000/")

        ##re-use detected language
        def lt_fun(row):
            if row.name % 5000 == 0: print(row.name)
            # print(row['lang'])
            transl = row['product_txt'] if row['lang'] == 'fr' else lt.translate(
                row['product_txt'], source=row['lang'], target="fr")
            return transl


        data['product_txt_transl'] = data.apply(lambda row: lt_fun(row), axis=1)

        ##save translations to csv (overwrites existing file)
        data.to_csv(csv_file)
    else:
        ##load translations from stored csv file
        data = pd.read_csv(csv_file)

    return data



In [623]:
##translate X train (by default load existing csv file with translations)
Rak_train = translate_txt(data = None, rerun = False, csv_file = '../data/processed/Rak_train_translations.csv')

##FIXME run translations for test as well


In [599]:
##Re-display phrases from RAW data after clean-up
# print(Xa_raw.index)

# display(Rak_train.loc[Xa_raw.index, :])


In [594]:
##Display random sample from clean-ed up data (explore residual text problems in phrases)
# print(dir(Rak_train_raw))
Xa = Rak_train.sample(10)
Xa_idx = Xa.index  ##save random index temporarily
# print(Xa.index)

display(Xa) 



,designation,description,productid,imageid,lang,product_txt,product_txt_len,product_txt_transl
9873,le guide des professions juridiques volume juridictions administrations organismes,edition p .,1798672941,1102381830,fr,le guide des professions juridiques volume juridictions administrations organismes . -//- edition p .,104,le guide des professions juridiques volume juridictions administrations organismes . -//- edition p .
84294,jjrc telecommande interaction intelligente dance light robot enfants longue veille jouet temps rouge ( version anglaise ),description : nom de l'article : jouet couleur : rouge ( version anglaise ) bleu ( version anglaise ) alimentation : batterie nicd . duree de lecture : jusqu'a minutes temps de charge : heures age applicable : adolescent ( ans ) mode de controle : telecommande materiau : plastique l'emballage inclus : x robot ( batterie incluse ) x emetteur x cable usb jjrc defenders est un robot incroyable qui peut vous apporter beaucoup de plaisir . il peut marcher et glisser comme un ¿ºtre humain basculer entre vitesse elevee / faible et augmenter / diminuer le volume . quand il danse il y a un eclairage led incroyable et une musique dynamique . plus interessant encore il peut entrer en mode combat et utiliser son bras pour tirer des missiles dans differentes directions . avec le bouton de demonstration automatique le robot peut effectuer automatiquement une serie d ¡ ¯ actions . remarque : l ¡ ¯ ecart des couleurs peut differer en raison de parametres de moniteur differents veuillez bien comprendre .,3925301162,1264333322,fr,jjrc telecommande interaction intelligente dance light robot enfants longue veille jouet temps rouge ( version anglaise ) . -//- description : nom de l'article : jouet couleur : rouge ( version anglaise ) bleu ( version anglaise ) alimentation : batterie nicd . duree de lecture : jusqu'a minutes temps de charge : heures age applicable : adolescent ( ans ) mode de controle : telecommande materiau : plastique l'emballage inclus : x robot ( batterie incluse ) x emetteur x cable usb jjrc defenders est un robot incroyable qui peut vous apporter beaucoup de plaisir . il peut marcher et glisser comme un ¿ºtre humain basculer entre vitesse elevee / faible et augmenter / diminuer le volume . quand il danse il y a un eclairage led incroyable et une musique dynamique . plus interessant encore il peut entrer en mode combat et utiliser son bras pour tirer des missiles dans differentes directions . avec le bouton de demonstration automatique le robot peut effectuer automatiquement une serie d ¡ ¯ actions . remarque : l ¡ ¯ ecart des couleurs peut differer en raison de parametres de moniteur differents veuillez bien comprendre .,1190,jjrc telecommande interaction intelligente dance light robot enfants longue veille jouet temps rouge ( version anglaise ) . -//- description : nom de l'article : jouet couleur : rouge ( version anglaise ) bleu ( version anglaise ) alimentation : batterie nicd . duree de lecture : jusqu'a minutes temps de charge : heures age applicable : adolescent ( ans ) mode de controle : telecommande materiau : plastique l'emballage inclus : x robot ( batterie incluse ) x emetteur x cable usb jjrc defenders est un robot incroyable qui peut vous apporter beaucoup de plaisir . il peut marcher et glisser comme un ¿ºtre humain basculer entre vitesse elevee / faible et augmenter / diminuer le volume . quand il danse il y a un eclairage led incroyable et une musique dynamique . plus interessant encore il peut entrer en mode combat et utiliser son bras pour tirer des missiles dans differentes directions . avec le bouton de demonstration automatique le robot peut effectuer automatiquement une serie d ¡ ¯ actions . remarque : l ¡ ¯ ecart des couleurs peut differer en raison de parametres de moniteur differents veuillez bien comprendre .
58358,portable ukulele support pliable en bois support pliable vertical guitare présentoir stand,portable ukulele support pliable en bois support p

In [553]:
##Display random sample of translations
Xb = Rak_train.loc[ Rak_train['lang'] != 'fr', :].sample(10)

# display(Xb) 



In [490]:
##Display and Document problematic cases
if False:
# if True:
    # display(Rak_train.iloc[Xa.index])
    # display(Rak_train.iloc[[2]]) 
    # display(Rak_train.iloc[[5191, 78115, 59683, 12525, 61380]]) 
    display(Rak_train.iloc[[52014, 5210, 29834]])   ##german
    display(Rak_train.iloc[[27011]])   ##english4835
    display(Rak_train.iloc[[4835]])   ##portuguese
    display(Rak_train.iloc[[30846]])   ##chinese (pinyin)
    # display(Rak_train.iloc[[10190, 79035, 48620, 42086, 78822, 16367, 61862, 76611, 63470]])  
    display(Rak_train.iloc[[69199, 42210]])   ##html tags & accents
    display(Rak_train.iloc[[56642, 2815, 26649, 63901, 34335]])   ##FIXME corrupted fr accents (¿ or ?), apostrophe (l?attacher)
    display(Rak_train.iloc[[84805, 39879, 84156, 77019]])   ##®, n°, [
    display(Rak_train.iloc[[29766]])   ##text mixed with numbers 'fr102'

In [413]:
##Concat strings

Rak_train['product_txt'] = Rak_train['designation'] + ' . -//- ' + Rak_train['description']

Rak_train['product_txt_len'] = Rak_train['product_txt'].apply(len)

# display(Rak_train.head())

In [368]:
##detect phrase language using 'langid' on product_txt
import langid

# langid.set_languages(langs=None)
langid.set_languages(langs=['fr', 'en', 'de', 'it', 'es', 'pt'])
Rak_train['lang'] = Rak_train['product_txt'].apply(lambda x: langid.classify(x)[0])



In [464]:
##translate using libretranslate (self-hosted process)
##NOTE: need to start process first
# libretranslate --update-models --load-only fr,en,es,de,it,pt
# libretranslate --load-only fr,en,es,de,it,pt

from libretranslatepy import LibreTranslateAPI
lt = LibreTranslateAPI("http://localhost:5000/")

##re-use detected language
def lt_fun(row):
    if row.name % 10000 == 0: print(row.name)
    # print(row['lang'])
    transl = row['product_txt'] if row['lang'] == 'fr' else lt.translate(
        row['product_txt'], source=row['lang'], target="fr")
    return transl


Rak_train['product_txt_transl'] = Rak_train.apply(lambda row: lt_fun(row), axis=1)
# display(Rak_train[slc])


In [465]:
##save translations to csv
Rak_train.to_csv('../data/processed/Rak_train_translations.csv')

In [482]:
##translate non-fr languages w/ 'translate' pkg
##doesn't work, hits limits

# from translate import Translator
# translator = Translator(to_lang="fr")

# ##fun to translate given 500 char limitation of 'translate' pkg
# def transl_fun(row):
#     print(row.name)
#     txt = row['product_txt']
#     transl = ""
#     if row['lang'] == 'fr': 
#         transl = txt
#     elif len(txt) <= 500: 
#         transl = translator.translate(txt)
#     else:
#         txt_tkn = nltk.tokenize.sent_tokenize(txt)
#         for phrase in txt_tkn:
#             transl = transl + " " + translator.translate(phrase)
    
#     return transl 


# # Rak_train.loc[:, 'product_txt_transl']
# # Rak_train['product_txt_transl'] = Rak_train.apply(transl_fun, axis=1)

In [483]:
# import translate
# help(Translator)
# help(translate.translate)
# print(translate.builtins.object)

**Rebalancing**

In [552]:
##data is already split into Training/Testing, no need to re-split
# print(Rak_train.shape)
# display(Rak_train.head(2))

RakX_train = Rak_train[['product_txt_transl']]
# print(RakX_train.shape)
# display(RakX_train.head(2))

# print(RakY_train_raw.shape)
# display(RakY_train_raw.head(2))
print('Classes échantillon originales :', pd.Series.value_counts(RakY_train_raw['prdtypecode']))



Classes échantillon originales : prdtypecode
2583    10209
1560    5073 
1300    5045 
2060    4993 
2522    4989 
1280    4870 
2403    4774 
2280    4760 
1920    4303 
1160    3953 
1320    3241 
10      3116 
2705    2761 
1140    2671 
2582    2589 
40      2508 
2585    2496 
1302    2491 
1281    2070 
50      1681 
2462    1421 
2905    872  
60      832  
2220    824  
1301    807  
1940    803  
1180    764  
Name: count, dtype: int64


In [557]:
##Oversampling (only on training data)
from imblearn.over_sampling import RandomOverSampler, SMOTEN

##SMOTEN
smo = SMOTEN(random_state=27732)
RakX_train_sm, Raky_train_sm = smo.fit_resample(RakX_train, RakY_train_raw)



In [584]:
print([i.shape for i in (RakX_train_sm, Raky_train_sm)])

print('\nClasses échantillon SMOTEN :', Raky_sm.value_counts())

# slc = slice(185856, 185876)
display(RakX_train_sm.head(10))
display(RakX_train_sm['product_txt_transl'].head())
# display(RakX_train_sm.loc[slc, :])

# display(Raky_train_sm.head(10))
# display(Raky_train_sm.loc[slc, :])


[(275643, 1), (275643, 1)]

Classes échantillon SMOTEN : prdtypecode
10             10209
40             10209
50             10209
60             10209
1140           10209
1160           10209
1180           10209
1280           10209
1281           10209
1300           10209
1301           10209
1302           10209
1320           10209
1560           10209
1920           10209
1940           10209
2060           10209
2220           10209
2280           10209
2403           10209
2462           10209
2522           10209
2582           10209
2583           10209
2585           10209
2705           10209
2905           10209
Name: count, dtype: int64


,product_txt_transl
0,olivia : notizbuch / page / punktraster / ca din / rosen-design
1,journal des arts ( le ) n ° du / / - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans . . -//-
2,grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style . -//- pilot style touch pen de marque speedlink est stylet ergonomique pour gamepad nintendo wii u . pour un confort optimal et une précision maximale sur le gamepad de la wii u : ce grand stylet hautement ergonomique est non seulement parfaitement adapté à votre main mais aussi très élégant . il est livré avec un support qui se fixe sans adhésif à l'arrière du gamepad caractéristiques : modèle : speedlink pilot style touch pen couleur : bleu ref . fabricant : compatibilité : gamepad nintendo wii u forme particulièrement ergonomique excellente tenue en main pointe à revêtement longue durée conçue pour ne pas abîmer l'écran tactile en bonus : support inclu pour gamepad
3,peluche donald - europe - disneyland ( marionnette à doigt ) . -//-
4,la guerre des tuques . -//- luc a des idées de grandeur . il veut organiser un jeu de guerre de boules de neige et s'arranger pour en être le vainqueur incontesté . mais sophie s'en mêle et chambarde tous ses plans . . .
5,afrique contemporaine n ° hiver - dossier japon / afrique . -//-
6,christof e : processus de formation sur la piste
7,conquérant sept cahier couverture polypro x mm pages seyès incolore . -//- conquerant classique cahier x mm seyès incolorecouverture en polypro pages agrafé papier de g / ( )
8,puzzle scooby-doo avec poster pieces . -//-
9,tente pliante pvc blanc - x - longueur : largeur : m blanc h . -//- tente pliante pro pvc gr / m² - x . que vous soyez un particulier pour votre jardin ou un professionnel pour stand commercial ou pour vos réceptions le barnum pro de m² sera vous combler . imaginez un stand robuste léger adaptable à chacun pliable et peu encombrant . . . le déploiement ultra-rapide et le réglage de la hauteur se font maintenant via des poignées d'indexage . nous avons également entièrement repensé les pièces de jonction et les coulissants : plus rigides et légères elles sont aussi maintenant % plus résistantes . la tente pliante pro de qualité professionnelle et de fabrication française est complètement adaptable et deviendra l'outil indispensable à votre activité . toujours en vue d'améliorer votre quotidien cette version du se veut encore plus résistante au temps . un traitement ignifugé répond aux exigences de sécurité en vigueur ( certifié ) et sa facilité d'entretien en fait un allié pour toutes les occasions . le produit de base est composé d'une structure d'un toit en bâche pvc gr / m² et d'une housse . après à vous de configurer vos façades comme vous le souhaitez . avantages facile à monter traitement anti feu façades aux choix caractéristiques coloris blanc pvc blanc gr / m² ignifugé indexation via poignées : blocage du stand sans effort . hauteurs réglables via indexeurs . croisillons : renfort interne et renforts externes inox . embases : fonte d'aluminium ( mm ) . anneaux d'haubanage . structure % aluminium / visserie en inox . pièces de jonctions renforcées ( polypropylène + % fibre de verre ) fils haute ténacité hydrofuges pied coulissant et système de guidage intégré pieds hexagonaux en aluminium ( épaisseur mm ) renforts de toit pvc finition du toit intégrant un biais sergé auto agrippant mm maintien du toit par auto agrippants encombrement refermé inférieur à dimensions x mâts hauteur du faîtage à hauteur sous bandeau à dimensions repliées x x


0    olivia : notizbuch / page / punktraster / ca din / rosen-design                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
1    journal des arts  ( le )  n °   du  /  /  - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans .  

**Tokenization**

In [590]:
##FIXME

##Tokenization
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier


##Stop Words
fr_stop_words = stopwords.words('french')
# print(fr_stop_words)


# Créer un vectorisateur 
##FIXME consider custom tokenizer and max_features
# regexp_tokenizer = RegexpTokenizer("[a-zA-ZÀÂÆÇÉÈÊËÎÏÔŒÙÛÜŸàâæçéèêëîïôœùûüÿ]{3,}") ##words with at least 3 characaters
vect_tfidf = TfidfVectorizer(    
    max_features=10000,
    stop_words=fr_stop_words
    # , tokenizer = regexp_tokenizer
)

# Mettre à jour la valeur de X_train_tfidf et X_test_tfidf
RakX_train_sm_tfidf = vect_tfidf.fit_transform(RakX_train_sm['product_txt_transl'])
# RakX_test_tfidf = vect_tfidf.transform()



InvalidParameterError: The 'tokenizer' parameter of TfidfVectorizer must be a callable or None. Got RegexpTokenizer(pattern='[a-zA-ZÀÂÆÇÉÈÊËÎÏÔŒÙÛÜŸàâæçéèêëîïôœùûüÿ]{3,}', gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL) instead.

In [574]:
##inspect tokenized data
print(RakX_train_sm_tfidf.shape)
print(RakX_train_sm_tfidf.toarray().shape)

##inspect vocabulary
tokenized_vocab = vect_tfidf.vocabulary_
print(len(tokenized_vocab))
print(tokenized_vocab)


(1, 1)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 1)>
  Coords	Values
  (0, 0)	1.0
1


In [564]:
##multiclass logistic regression
from sklearn.linear_model import LogisticRegression

# Modèle Multiclass (LogisticRegression)
clf = LogisticRegression(max_iter=2000)
clf.fit(RakX_train_sm_tfidf, Raky_train_sm)
# y_pred = clf.predict(X_test)


C:\Users\geek\anaconda3\envs\proj_rakuten\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Found input variables with inconsistent numbers of samples: [1, 275643]

In [505]:
##manual tokenizer
##need to use sklearn version instead
regexp_tokenizer = RegexpTokenizer("[a-zA-ZÀÂÆÇÉÈÊËÎÏÔŒÙÛÜŸàâæçéèêëîïôœùûüÿ]{3,}") ##words with at least 3 characaters

# Calculer les tokens
# tokens = regexp_tokenizer.tokenize(txt)
# Rak_train['designation_l'] = Rak_train['designation'].apply(lambda X: regexp_tokenizer.tokenize(X))
# Rak_train['description_l'] = Rak_train['description'].apply(lambda X: regexp_tokenizer.tokenize(X))
# Rak_train = Rak_train.drop(columns = ['designation_l', 'description_l'])

Rak_train['product_tokens'] = Rak_train['product_txt'].apply(lambda x: regexp_tokenizer.tokenize(x))

# Rak_train = Rak_train.drop(columns=['product_tokens'])

# Afficher les tokens
display(Rak_train.head(1))

,designation,description,productid,imageid,lang,product_txt,product_txt_len,product_txt_transl
0,olivia : personalisiertes notizbuch / seiten / punktraster / ca din / rosen-design,,3804725264,1263597046,de,olivia : personalisiertes notizbuch / seiten / punktraster / ca din / rosen-design . -//-,101,olivia : notizbuch / page / punktraster / ca din / rosen-design


**Merge Lists**

In [355]:
##concatenation done upstream
# Rak_train['design_descr_l'] = 
# Xb = Rak_train.iloc[[0,1,2]]
# display(Xb)
# Xb['d'] = Xb.apply(lambda row: row['designation'], axis=1)
# display(Rak_train.iloc[[0,1,2]].apply(lambda row: row['designation'], axis=1))

# Rak_train.iloc[[0,1,2]].apply(lambda row: row['designation_l'] + row['description_l'], axis=1)

# Rak_train['desig_descr_l'] = Rak_train.apply(lambda row: row['designation_l'].extend( row['description_l'] ), axis=1)
# display(Rak_train)

In [357]:
#####################
##test detect language w/ langdetect
##not very accurate
from langdetect import detect

# print(detect("Geeksforgeeks is a computer science portal for geeks"))
# print(detect("Geeksforgeeks - это компьютерный портал для гиков"))
# print(detect("Geeksforgeeks es un portal informático para geeks"))
# print(detect("Geeksforgeeks este un portal informatic pentru geeks"))
# print(detect("Geeksforgeeks是面向极客的计算机科学门户"))
# print(detect("Geeksforgeeks geeks के लिए एक कंप्यूटर विज्ञान पोर्टल है"))
# print(detect("Geeksforgeeksは、ギーク向けのコンピューターサイエンスポータルです。"))


# print(detect(Rak_train.iloc[[0,1,2], 1]))
##mixed col detection
slc = slice(0,20)
# print(Rak_train[slc].apply(lambda row: detect(row['designation']) if row['description'] == '' else detect(row['description']), axis=1))
##only descr-based detection
# print(Rak_train[10:20].apply(lambda row: '' if row['description'] == '' else detect(row['description']), axis=1))
##product_txt based detection
print(Rak_train.loc[slc, 'product_txt'].apply(lambda x: detect(x)))

# display(Rak_train[slc])


# help(detect)

0     it
1     fr
2     fr
3     fr
4     fr
5     fr
6     de
7     fr
8     it
9     fr
10    en
11    fr
12    en
13    fr
14    fr
15    fr
16    fr
17    en
18    en
19    fr
20    fr
Name: product_txt, dtype: object


In [328]:
##NLTK
import nltk
# nltk.download('crubadan')
import pycountry
from nltk.stem import SnowballStemmer

# phrase_one = "good morning"
# phrase_two = "goeie more"
df_phrases = pd.DataFrame({'a': ["good morning", "goeie more", "bonjour l'amour", "guten tag"]})

tc = nltk.classify.textcat.TextCat() 
# guess_one = tc.guess_language(phrase_one)
# guess_two = tc.guess_language(phrase_two)
guesses = df_phrases.apply(lambda row: tc.guess_language(row['a']), axis=1)

# print(guess_one)
# print(guess_two)
# print(guesses)

# help(tc.guess_language)
# help(nltk.classify.textcat.TextCat)
# dir(guess_one)
# guess_one_name = pycountry.languages.get(alpha_3=guess_one).name
# guess_two_name = pycountry.languages.get(alpha_3=guess_two).name
# print(guess_one_name)
# print(guess_two_name)
# print(pycountry.languages.get(alpha_3='ilo').name)


In [333]:
slc = slice(0,20)
##mixed col detection
# print(Rak_train[slc].apply(lambda row: tc.guess_language(row['designation']) if row['description'] == '' else 
#                             tc.guess_language(row['description']), axis=1))
##only descr-based detection
# print(Rak_train[slc].apply(lambda row: '' if row['description'] == '' else tc.guess_language(row['description']), axis=1))
##product_txt based detection
print(Rak_train.loc[slc, 'product_txt'].apply(lambda x: tc.guess_language(x)))

# display(Rak_train[slc])


0     deu 
1     fra 
2     fra 
3     fra 
4     fra 
5     fra 
6     deu 
7     fra 
8     eng 
9     fra 
10    eng 
11    fra 
12    eng 
13    fra 
14    fra 
15    fra 
16    fra 
17    eng 
18    eng 
19    eng 
20    fra 
Name: product_txt, dtype: object


In [339]:
##LANGID 
##seems to work best (and has best perf as well)
import langid
langid.set_languages(langs=None)
# langid.set_languages(langs=['fr', 'en', 'ru', 'ro', 'ja', 'hi', 'zh', 'es', 'de'])

texts = [
    "Geeksforgeeks is a computer science portal for geeks",
    "Geeksforgeeks - это компьютерный портал для гиков",
    "Geeksforgeeks este un portal informatic pentru geeks",
    "Geeksforgeeks es un portal informático para geeks",
    "Geeksforgeeks是面向极客的计算机科学门户",
    "Geeksforgeeks ist un portal informatichen fur geeks",
    "Geeksforgeeks geeks के लिए एक कंप्यूटर विज्ञान पोर्टल है",
    "Geeksforgeeksは、ギーク向けのコンピューターサイエンスポータルです。"
]

for t in texts:
    #print(langid.classify(t))
    pass

# help(langid)
# help(langid.set_languages)




In [429]:
langid.set_languages(langs=None)
# langid.set_languages(langs=['fr', 'en', 'es', 'de', 'it', 'pt'])


slc = slice(0,20)
##mixed col detection
# print(Rak_train[slc].apply(lambda row: langid.classify(row['designation'])[0] if row['description'] == '' else 
#                             langid.classify(row['description'])[0], axis=1))
##only descr-based detection
# print(Rak_train[slc].apply(lambda row: '' if row['description'] == '' else langid.classify(row['description']), axis=1))
##product_txt based detection
print(Rak_train.loc[slc, 'product_txt'].apply(lambda x: langid.classify(x)[0]))


# display(Rak_train.loc[slc])  ##[] subsetting vs .loc yields different resutls for the same slice


0     de
1     fr
2     fr
3     fr
4     fr
5     fr
6     de
7     fr
8     fr
9     fr
10    en
11    fr
12    en
13    fr
14    fr
15    fr
16    fr
17    en
18    en
19    fr
20    fr
Name: product_txt, dtype: object


In [353]:
slc = slice(0,20)

print(Rak_train['lang'].value_counts()[0:25])

# display(Rak_train[slc])
# display(Rak_train.loc[Rak_train['lang'] == 'it'].sample(30))


lang
fr    70266
en    8496 
it    784  
es    738  
de    687  
nl    501  
pt    252  
ro    194  
ca    174  
id    169  
oc    158  
br    154  
eu    147  
eo    143  
no    132  
da    131  
pl    114  
la    106  
cy    93   
lb    88   
et    81   
jv    80   
sw    75   
sv    74   
tl    68   
Name: count, dtype: int64


In [376]:
##test translate language w/ googletrans
##can't extract translation
from googletrans import Translator, constants

# init the Google API translator
translator = Translator()

# translate a spanish text to english text (by default)
translation = translator.translate("Hola Mundo")
# print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")
print(dir(translation))
# for i in dir(translation): print(translation[i])
# print(translation.cr_origin)
print(translation)

['__await__', '__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'cr_await', 'cr_code', 'cr_frame', 'cr_origin', 'cr_running', 'cr_suspended', 'send', 'throw']
<coroutine object Translator.translate at 0x0000028F97858F40>


C:\Users\geek\AppData\Local\Temp\ipykernel_32460\621738557.py:9: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  translation = translator.translate("Hola Mundo")


In [377]:
# help(translator)

In [430]:
##test translate language w/ 'translate' pkg
from translate import Translator
translator = Translator(to_lang="fr")
# translation = translator.translate("This is a pen.")
translation = translator.translate("Asta-i un obiect foarte frumos.")
print(translation)


x = '''I am using google translate api to translate some text.

The package is from deep_translator import GoogleTranslator.

However, every time it would return the error: --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated

Does anyone know how to solve this? Is it possible to have a paid version of api to expand the maximum characters?

Thanks a lot!'''

print(len(x))
x = nltk.tokenize.sent_tokenize(x)
for sentence in x:
    print(sentence) #send each sentence to translator

x = Rak_train.loc[9, 'product_txt']
print(len(x))

x_tkn = nltk.tokenize.sent_tokenize(x)
# print(x_tkn)

# for sentence in x_tkn:
#     print(sentence) #send each sentence to translator


C'est un très bel objet.
389
I am using google translate api to translate some text.
The package is from deep_translator import GoogleTranslator.
However, every time it would return the error: --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated

Does anyone know how to solve this?
Is it possible to have a paid version of api to expand the maximum characters?
Thanks a lot!
2073


In [436]:
slc = slice(45,49)
display(Rak_train[slc])

##fun to translate given 500 char limitation of 'translate' pkg
def transl_fun(row):
    print(row.name)
    txt = row['product_txt']
    transl = ""
    if row['lang'] == 'fr': 
        transl = txt
    elif len(txt) <= 500: 
        transl = translator.translate(txt)
    else:
        txt_tkn = nltk.tokenize.sent_tokenize(txt)
        for phrase in txt_tkn:
            transl = transl + " " + translator.translate(phrase)
    
    return transl 


print(Rak_train[slc].apply(transl_fun, axis=1))
# display(Rak_train[slc])


# print(Rak_train[slc].apply(lambda row: translator.translate(row['description']) if row['lang'] != 'fr' else 
#                             row['description'], axis=1))
# display(Rak_train[slc])


# Rak_train[slc].apply(lambda row: row['product_txt'] if row['lang'] == 'fr' else 
#                                                          translator.translate(row['product_txt']), axis=1)
# display(Rak_train[slc])


,designation,description,productid,imageid,lang,product_txt,product_tokens,product_txt_len
45,rideau a oeillets plastique polyester uni essentiel lin x cm,rêt-à-poser ce rideau s'installe facilement et vous fera gagner du temps composition : % polyester ce rideau vous protégera des regards extérieurs et ne laissera pas passer la lumière dimension : x cm finition : oeillets ronds plastique plusieurs coloris dans cette collection ainsi que des galettes de chaise des housses de coussin des nappes pour donner un coté original à votre interieur,2333136838,1136619798,fr,rideau a oeillets plastique polyester uni essentiel lin x cm . -//- rêt-à-poser ce rideau s'installe facilement et vous fera gagner du temps composition : % polyester ce rideau vous protégera des regards extérieurs et ne laissera pas passer la lumière dimension : x cm finition : oeillets ronds plastique plusieurs coloris dans cette collection ainsi que des galettes de chaise des housses de coussin des nappes pour donner un coté original à votre interieur,"[rideau, oeillets, plastique, polyester, uni, essentiel, lin, rêt, poser, rideau, installe, facilement, vous, fera, gagner, temps, composition, polyester, rideau, vous, protégera, des, regards, extérieurs, laissera, pas, passer, lumière, dimension, finition, oeillets, ronds, plastique, plusieurs, coloris, dans, cette, collection, ainsi, que, des, galettes, chaise, des, housses, coussin, des, nappes, pour, donner, coté, original, votre, interieur]",483
46,vassivière -en-limousin . le jardin art et lieu de mémoire,,444108636,1010065513,fr,vassivière -en-limousin . le jardin art et lieu de mémoire . -//-,"[vassivière, limousin, jardin, art, lieu, mémoire]",68
47,shin masoukishin panzer warfare ( import jap ),,8870838,868167157,en,shin masoukishin panzer warfare ( import jap ) . -//-,"[shin, masoukishin, panzer, warfare, import, jap]",56
48,dragon ball z - serie - - francaise,,73334276,844838682,fr,dragon ball z - serie - - francaise . -//-,"[dragon, ball, serie, francaise]",45


In [444]:
##test libretranslate

from libretranslatepy import LibreTranslateAPI

# lt = LibreTranslateAPI("https://libretranslate.com/")
lt = LibreTranslateAPI("http://localhost:5000/")

print(lt.translate("LibreTranslate es impresionante!", "auto", "fr"))
# LibreTranslate es impresionante!

print(lt.detect("Hello World"))
# [{"confidence": 0.6, "language": "en"}]

# print(lt.languages())
# print(dir(lt))

LibreTrail est impressionnant !
[{'confidence': 90.0, 'language': 'en'}]


In [451]:
slc = slice(0,30)
print(Rak_train[slc].apply(lambda row: lt.detect(row['product_txt']), axis=1))


0     [{'confidence': 86.0, 'language': 'it'}, {'confidence': 14.0, 'language': 'es'}]
1     [{'confidence': 100.0, 'language': 'fr'}]                                       
2     [{'confidence': 100.0, 'language': 'fr'}]                                       
3     [{'confidence': 100.0, 'language': 'fr'}]                                       
4     [{'confidence': 100.0, 'language': 'fr'}]                                       
5     [{'confidence': 100.0, 'language': 'fr'}]                                       
6     [{'confidence': 0.0, 'language': 'en'}]                                         
7     [{'confidence': 100.0, 'language': 'fr'}]                                       
8     [{'confidence': 29.0, 'language': 'it'}]                                        
9     [{'confidence': 100.0, 'language': 'fr'}]                                       
10    [{'confidence': 100.0, 'language': 'en'}]                                       
11    [{'confidence': 100.0, 'language': 'f

In [462]:
##NOTE: need to start process first
# libretranslate --update-models --load-only fr,en,es,de,it,pt
# libretranslate --load-only fr,en,es,de,it,pt

from libretranslatepy import LibreTranslateAPI
lt = LibreTranslateAPI("http://localhost:5000/")


slc = slice(0,10)
##autodetect
# print(Rak_train[slc].apply(lambda row: row['product_txt'] if row['lang'] == 'fr' else 
#                      lt.translate(row['product_txt'], "auto", "fr"), axis=1))


##re-use detected language
def lt_fun(row):
    # print(row['lang'])
    transl = row['product_txt'] if row['lang'] == 'fr' else lt.translate(
        row['product_txt'], source=row['lang'], target="fr")
    return transl


print(Rak_train[slc].apply(lambda row: lt_fun(row), axis=1))

display(Rak_train[slc])


0    olivia : notizbuch / page / punktraster / ca din / rosen-design                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

,designation,description,productid,imageid,lang,product_txt,product_tokens,product_txt_len
0,olivia : personalisiertes notizbuch / seiten / punktraster / ca din / rosen-design,,3804725264,1263597046,de,olivia : personalisiertes notizbuch / seiten / punktraster / ca din / rosen-design . -//-,"[olivia, personalisiertes, notizbuch, seiten, punktraster, din, rosen, design]",101
1,journal des arts ( le ) n ° du / / - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans .,,436067568,1008141237,fr,journal des arts ( le ) n ° du / / - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans . . -//-,"[journal, des, arts, art, son, marche, salon, art, asiatique, paris, jacques, barrere, francois, perrier, reforme, des, ventes, aux, encheres, publiques, sna, fete, ses, cent, ans]",215
2,grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style,pilot style touch pen de marque speedlink est stylet ergonomique pour gamepad nintendo wii u . pour un confort optimal et une précision maximale sur le gamepad de la wii u : ce grand stylet hautement ergonomique est non seulement parfaitement adapté à votre main mais aussi très élégant . il est livré avec un support qui se fixe sans adhésif à l'arrière du gamepad caractéristiques : modèle : speedlink pilot style touch pen couleur : bleu ref . fabricant : compatibilité : gamepad nintendo wii u forme particulièrement ergonomique excellente tenue en main pointe à revêtement longue durée conçue pour ne pas abîmer l'écran tactile en bonus : support inclu pour gamepad,201115110,938777978,fr,grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style . -//- pilot style touch pen de marque speedlink est stylet ergonomique pour gamepad nintendo wii u . pour un confort optimal et une précision maximale sur le gamepad de la wii u : ce grand stylet hautement ergonomique est non seulement parfaitement adapté à votre main mais aussi très élégant . il est livré avec un support qui se fixe sans adhésif à l'arrière du gamepad caractéristiques : modèle : speedlink pilot style touch pen couleur : bleu ref . fabricant : compatibilité : gamepad nintendo wii u forme particulièrement ergonomique excellente tenue en main pointe à revêtement longue durée conçue pour ne pas abîmer l'écran tactile en bonus : support inclu pour gamepad,"[grand, stylet, ergonomique, bleu, gamepad, nintendo, wii, speedlink, pilot, style, pilot, style, touch, pen, marque, speedlink, est, stylet, ergonomique, pour, gamepad, nintendo, wii, pour, confort, optimal, une, précision, maximale, sur, gamepad, wii, grand, stylet, hautement, ergonomique, est, non, seulement, parfaitement, adapté, votre, main, mais, aussi, très, élégant, est, livré, avec, support, qui, fixe, sans, adhésif, arrière, gamepad, caractéristiques, modèle, speedlink, pilot, style, touch, pen, couleur, bleu, ref, fabricant, compatibilité, gamepad, nintendo, wii, forme, particulièrement, ergonomique, excellente, tenue, main, pointe, revêtement, longue, durée, conçue, pour, pas, abîmer, écran, tactile, bonus, support, inclu, pour, gamepad]",781
3,peluche donald - europe - disneyland ( marionnette à doigt ),,50418756,457047496,fr,peluche donald - europe - disneyland ( marionnette à doigt ) . -//-,"[peluche, donald, europe, disneyland, marionnette, doigt]",71
4,la guerre des tuques,luc a des idées de grandeur . il veut organiser un jeu de guerre de boules de neige et s'arranger pour en être le vainqueur incontesté . mais sophie s'en mêle et chambarde tous ses plans . . .,278535884,1077757786,fr,la guerre des tuques . -//- luc a des idées de grandeur . il veut organiser un jeu de guerre de boules de neige et s'arranger pour en être le vainqueur incontesté . mais sophie s'en mêle et chambarde tous ses plans . . .,"[guerre, des, tuques, luc, de

**Remove Stopwords**

In [445]:
fr_stop_words = set(stopwords.words('french'))
# en_stop_words = set(stopwords.words('english'))
# de_stop_words = set(stopwords.words('german'))
print(fr_stop_words)
# print(en_stop_words)
# print(de_stop_words)


{'aurai', 'eus', 'avaient', 'en', 'est', 'fussiez', 'soyons', 'j', 'avions', 'serait', 'c', 'une', 't', 'aux', 'aurez', 'auront', 'ayant', 'ayez', 'sommes', 'eussent', 'auras', 'du', 'eussions', 'ayante', 'au', 'm', 'ayants', 'notre', 'as', 'pour', 'aurons', 'eux', 'ses', 'étée', 'leur', 'qui', 'sera', 'ait', 'étées', 'soient', 'seront', 'on', 'sont', 'aurais', 'eut', 'était', 'aura', 'étantes', 'aurait', 'fûmes', 'avez', 'avec', 'eûtes', 'eûmes', 'vous', 'auraient', 'toi', 'eu', 'y', 'ta', 'un', 'me', 'même', 'soit', 'ton', 'aies', 'ou', 'étante', 'aviez', 'sois', 'sa', 'soyez', 'par', 'tes', 'aient', 'ont', 'eurent', 'les', 'étions', 'serais', 'lui', 'étant', 'avait', 'de', 'l', 's', 'serons', 'la', 'furent', 'vos', 'eusses', 'fûtes', 'fusses', 'je', 'nos', 'à', 'étais', 'il', 'aurions', 'ne', 'serai', 'fussent', 'avons', 'es', 'que', 'serez', 'fussions', 'fus', 'des', 'serions', 'eues', 'suis', 'se', 'nous', 'ce', 'tu', 'et', 'ayantes', 'eue', 'le', 'seriez', 'seras', 'mon', 'moi', 

In [165]:
# définir la fonction stop_words_filtering
def stop_words_filtering(mots) : 
    tokens = []
    for mot in mots:
        if mot not in stop_words:
            tokens.append(mot)
    return tokens


In [409]:
xa = ""
xa += "txt"
print(xa)

txt
